# Programming constructs as data structures

- Data literals: e.g. 3.1415, "Hello world".
- Arithmetic expressions: `+`, `-`, `*`, `/`.
- Variables:
    - Referencing variables: `pi`
    - Assignment of variables: `pi = 3.1415`
- Logical conditions: `pi < 3`
- If/Else branching

# Everything is an instance of `Expr` class.

In [1]:
#
# Expr base class
#
class Expr:
    def evaluate(self, context):
        raise Exception("Not implement")
    def __str__(self):
        return "__str__() is not implemented."

In [2]:
#
# Literal
#

class Literal(Expr):
    def __init__(self, value):
        self.value = value
        
    def evaluate(self, context):
        return self.value
    
    def __str__(self):
        return "%s" % self.value

In [3]:
###
# 3.1415
###

e = Literal(3.1415)
print("%s => %s" % (e, e.evaluate(None)))

3.1415 => 3.1415


In [4]:
#
# Arith - mathematical operations over two expressions
#

class Arith(Expr):
    math_ops = {
        'plus': lambda x,y:x + y,
        'sub': lambda x, y: x - y,
        'mult': lambda x, y: x * y,
        'div': lambda x, y: x / y
    }
    
    def __init__(self, left:Expr, op:str, right:Expr):
        self.left = left
        self.op = op
        self.right = right
    
    def evaluate(self, context):
        if self.op in self.math_ops:
            f = self.math_ops[self.op]
            x = self.left.evaluate(context)
            y = self.right.evaluate(context)
            return f(x, y)
        else:
            raise Exception("Unknown operator: %s" % self.op)
            
    def __str__(self):
        return "(%s %s %s)" % (self.op, self.left, self.right)
    

In [5]:
####
# Area of a circle with radius of 10.2
# 3.1415 * (10.2 * 10.2)
####

e= Arith(
    Literal(3.1415), 
    'mult',
    Arith(
        Literal(10.2),
        'mult',
        Literal(10.2)))
print("%s\n=> %s" % (e, e.evaluate(None)))

(mult 3.1415 (mult 10.2 10.2))
=> 326.84166


# Definition of a context

> A context is a mapping of variable (names) to their data values.

This is also known as a _scope_ (refer to _Programming Languages_).

In [8]:
#
# Manually create a context
#
C = {
    "pi": 3.1415,
    "radius": 10.2
}

In [9]:
#
# Variable
#

class Var(Expr):
    def __init__(self, name):
        self.name = name
        
    def evaluate(self, context):
        if self.name in context:
            return context[self.name]
        else:
            raise Exception("Variable is not defined in context: %s" % self.name)
    
    def __str__(self):
        return self.name

In [11]:
#####
# Looking up the value of pi by variable reference:
# pi
#####
e = Var("pi")
print("%s\n=> %s" % (e, e.evaluate(C)))

pi
=> 3.1415


In [14]:
####
# Calculate the area of the circle using variables
# pi * radius * radius
####

e = Arith(Var("pi"), 'mult', Arith(Var("radius"), 'mult', Var("radius")))
print("%s\n=> %s" % (e, e.evaluate(C)))

(mult pi (mult radius radius))
=> 326.84166


In [18]:
#
# Assignment to variables
#

class Assign(Expr):
    def __init__(self, name:str, expr:Expr):
        self.name = name
        self.expr = expr
        
    def evaluate(self, context):
        value = self.expr.evaluate(context)
        context[self.name] = value
        return value
        
    def __str__(self):
        return "%s = %s" % (self.name, self.expr)

In [20]:
#####
# i = 0
#####
e = Assign("i", Literal(0))
print("%s\n=> %s" % (e, e.evaluate(C)))

i = 0
=> 0


In [21]:
C

{'pi': 3.1415, 'radius': 10.2, 'i': 0}

In [22]:
#####
# i = i + 1
#####
e = Assign("i", Arith(Var("i"), 'plus', Literal(1)))
print("%s\n=> %s" % (e, e.evaluate(C)))

i = (plus i 1)
=> 1


In [23]:
C

{'pi': 3.1415, 'radius': 10.2, 'i': 1}

# Branching

- Logical condition expressions
- IfElse expressions

In [24]:
#
# Conditions
#

logical_cond = {
    'lt': lambda x,y: x < y,
    'gt': lambda x,y: x > y,
    'ne': lambda x,y: not(x == y),
    'le': lambda x,y: x <= y,
    'ge': lambda x,y: x >= y,
    'eq': lambda x,y: x == y,
    'and': lambda x,y: x and y,
    'or': lambda x,y: x or y
}

class Cond(Arith):
    def evaluate(self, context):
        x = self.left.evaluate(context)
        y = self.right.evaluate(context)
        return logical_cond[self.op](x, y)

In [25]:
######
# radius > 10
######
print("C = ", C)
e = Cond(Var("radius"), 'gt', Literal(10))
print("%s\n=> %s" % (e, e.evaluate(C)))

C =  {'pi': 3.1415, 'radius': 10.2, 'i': 1}
(gt radius 10)
=> True


In [45]:
#
# Branching expression with if-else
#

class IfElse(Expr):
    def __init__(self, cond:Cond, if_expr:Expr, else_expr:Expr):
        self.cond = cond
        self.if_expr = if_expr
        self.else_expr = else_expr
    
    def evaluate(self, context):
        if self.cond.evaluate(context):
            return self.if_expr.evaluate(context)
        else:
            return self.else_expr.evaluate(context)
        
    def __str__(self):
        return "if %s {\n%s\n} else {\n%s\n}" % (self.cond, self.if_expr, self.else_expr)

In [46]:
#######
# if radius > 10 then
#   "Greater than 10"
# else
#   "not bigger than 10"
#######

e = IfElse(
    Cond(Var("radius"), 'gt', Literal(100)),
    Literal("Greater than 10"),
    Literal("not bigger than 10")
)

print("C = ", C)
print("%s\n=> %s" % (e, e.evaluate(C)))

C =  {'pi': 3.1415, 'radius': 10.2, 'i': 1}
if (gt radius 100) {
Greater than 10
} else {
not bigger than 10
}
=> not bigger than 10


# Blocks

- can hold multiple (in a sequence) arbitrary expressions
- a block of expressions is evaluated to be the result of its _last_ expression.

In [28]:
class Block(Expr):
    def __init__(self, expr_list):
        self.expr_list = expr_list
        
    def evaluate(self, ctx):
        result = None
        for e in self.expr_list:
            result = e.evaluate(ctx)
        return result
    
    def __str__(self):
        return "\n".join(str(e) for e in self.expr_list)

In [29]:
####
# i = 10
# j = 20
# i + j
####

e = Block([
    Assign("i", Literal(10)),
    Assign("j", Literal(20)),
    Arith(Var("i"), "plus", Var("j"))
])
print("%s\n=> %s" % (e, e.evaluate({})))

i = 10
j = 20
(plus i j)
=> 30


# While loops

- constructed by a condition and a block
- evaluation returns the last expression that has been evaluated during the loop iteration.

In [30]:
class WhileLoop(Expr):
    def __init__(self, cond:Cond, body:Expr):
        self.cond = cond
        self.body = body
    
    def evaluate(self, context):
        result = None
        while self.cond.evaluate(context):
            result = self.body.evaluate(context)
        return result
    
    def __str__(self):
        return "while %s {\n%s\n}" % (self.cond, self.body)

In [36]:
######
# i = 0
# sum = 0
# while i <= 1000:
#   sum = sum + i
#   i = i + 1
# sum
######

e = Block([
    Assign("i", Literal(0)),
    Assign("sum", Literal(0)),
    WhileLoop(
        Cond(Var("i"), "le", Literal(1000)),
        Block([
            Assign("sum", Arith(Var("sum"), "plus", Var("i"))),
            Assign("i", Arith(Var("i"), "plus", Literal(1)))
        ])),
    Var("sum")
])

print("%s\n=> %s" % (e, e.evaluate({})))

i = 0
sum = 0
while (le i 1000) {
sum = (plus sum i)
i = (plus i 1)
}
sum
=> 500500


# Function Declaration

Constructed from:

- A name:str, this name will be used to create a new entry in the context (like Assign)
- A list of parameter names (which are strings)
- A body:Expr


In [37]:
class FuncDecl(Expr):
    def __init__(self, name:str, param_names:[str], body:Expr):
        self.name = name
        self.param_names = param_names
        self.body = body
    
    def evaluate(self, context):
        context[self.name] = self
        return None
    
    def __str__(self):
        return "function %s(%s) {\n%s\n}" % (self.name,
                                            ",".join(self.param_names),
                                            self.body)

In [39]:
######
# def add(i, j):
#   return i + j
######

e = FuncDecl("add", ["i", "j"], Arith(Var("i"), "plus", Var("j")))
print("%s\n=> %s" % (e, e.evaluate({})))

function add(i,j) {
(plus i j)
}
=> None


# Function Invocation

Constructed:

- name of function as a string
- a list of expressions as arguments

Evaluation:

- Need to evaluate each argument.
- **Need to create a sub-context by copying the existing context, and create additional symbol bindings
that bind parameter names to the respective arguments**
- Evaluate the body of the function.

In [47]:
class FuncInvoke(Expr):
    def __init__(self, name:str, arg_list:[Expr]):
        self.name = name
        self.arg_list = arg_list
    
    def evaluate(self, context):
        f = context[self.name]
        
        if not len(f.param_names) == len(self.arg_list):
            raise Exception("Incorrect number of arguments.")
        
        arg_values = [e.evaluate(context) for e in self.arg_list]
        subctx = context.copy()
        subctx.update(zip(f.param_names, arg_values))
        
        return f.body.evaluate(subctx)
    
    def __str__(self):
        return "%s(%s)" % (self.name, ",".join(str(e) for e in self.arg_list))

In [48]:
######
# def f(i, j):
#   i + j
#
# f(10, 20)
######

e = Block([
    FuncDecl("f", ["i", "j"], Arith(Var("i"), "plus", Var("j"))),
    FuncInvoke("f", [Literal(10), Literal(20)])
])

print("%s\n=> %s" % (e, e.evaluate({})))

function f(i,j) {
(plus i j)
}
f(10,20)
=> 30


# Recursive implementation of Fibonacci numbers

```
def fib(n):
  if n <= 2:
     1
  else:
     fib(n-1) + fib(n-2)
     
fib(10)
```

In [50]:
e = Block([
    FuncDecl("fib",
             ["n"],
            IfElse(
                Cond(Var("n"), "le", Literal(2)),
                Literal(1),
                Arith(
                    FuncInvoke("fib", [Arith(Var("n"), "sub", Literal(1))]),
                    "plus",
                    FuncInvoke("fib", [Arith(Var("n"), "sub", Literal(2))])
                ))),
    FuncInvoke("fib", [Literal(20)])
])

print("%s\n=> %s" % (e, e.evaluate({})))

function fib(n) {
if (le n 2) {
1
} else {
(plus fib((sub n 1)) fib((sub n 2)))
}
}
fib(20)
=> 6765
